In [ ]:
#supponiamo di avere tre pazienti con EHRs di lunghezze diverse 
#vogliamo applicare CBOW considerando il contesto K (user-defined) separatamente per ogni "sentence",
#cioe' separatamente per ogni paziente
#Creiamo one list with different arrays storing the tokens of the medical terms
# one array for each patient
P1 = "A wiki is run using wiki software, otherwise known as a wiki engine.\
         A wiki engine is a type of content management system, but it differs\
         from most other such systems, including blog software, in that the\
         content is created without any defined owner or leader, and wikis have\
         little implicit structure, allowing structure to emerge according to the\
         needs of the users."

P2 = "The online encyclopedia project Wikipedia is by far the most popular wiki-based\
         website, and is one of the most widely viewed sites of any kind in the world,\
         having been ranked in the top ten since 2007."

P3 = "Wikipedia is not a single wiki but rather a collection of hundreds of wikis,\
         one for each language. There are tens of thousands of other wikis in use, both\
         public and private, including wikis functioning as knowledge management resources,\
         notetaking tools, community websites and intranets. The English-language Wikipedia\
         has the largest collection of articles; as of September 2016, it had over five\
         million articles."

##ARRAY with tokens from patients
H = [P1.split(), P2.split(), P3.split()]

##Fun1: creiamo il nostro dictionary (associamo univocamente un token a un intero)
##Input: array of sentences
def create_dict(array_sentences): 
    word_to_ix = {}
    ix_to_word = {}
    
    for _, j in enumerate(array_sentences):
        for word in j:
            if word not in word_to_ix:
                word_to_ix[word] = len(word_to_ix)
                ix_to_word[len(ix_to_word)] = word
    return word_to_ix, ix_to_word

word_to_ix, ix_to_word = create_dict(H)

data = [[word_to_ix[mt] for mt in a] for _, a in enumerate(H)]

vocab_size = len(word_to_ix)
embedding_dim = 10
kernel_size = 3
k_max = 10

model = ehrModel(vocab_size, embedding_dim, kernel_size, k_max)

optimizer = torch.optim.Adam(
    model.parameters(), lr=0.001, weight_decay=1e-5)

##in order to compute the gradient with respect to both the input and the output:
def mse_loss(input, target):
    return torch.sum((input - target)**2) / input.data.nelement()

for epoch in range(5):
    for i, d in enumerate(data):
        print("vect {0:1d}".format(i))
        in_vect, out, encoded_vect = model(torch.tensor(d))
    
        MSE_loss = mse_loss(in_vect, out)
        print("MSE loss: {0}".format(MSE_loss))
    
        optimizer.zero_grad()
        MSE_loss.backward()
        optimizer.step()